# Reduce size of geojson files

To reduce the file size and so have faster-loading maps, this notebook does two things to the features in a geojson file:

+ Reduce the precision of the coordinates (from typically 15 decimal places, e.g. [-0.099722888517677, 51.5167693121822]).
+ Remove any field except coordinates and the data for plotting, in this case: LSOA11NM.

## Setup

In [1]:
# For importing geojson:
import json

# For saving new geojson:
from geojson import FeatureCollection

import copy

## Import the geojson

In [3]:
with open('../data_geojson/ons_data/LSOA_(Dec_2011)_Boundaries_Super_Generalised_Clipped_(BSC)_EW_V3.geojson') as f:
    geojson_ew = json.load(f)

Check which fields are stored in the geojson:

In [4]:
geojson_ew.keys()

dict_keys(['type', 'name', 'crs', 'features'])

In [5]:
geojson_ew['type']

'FeatureCollection'

In [6]:
geojson_ew['name']

'LSOA_(Dec_2011)_Boundaries_Super_Generalised_Clipped_(BSC)_EW_V3'

In [7]:
geojson_ew['crs']

{'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}}

The above fields can remain as they are necessary and/or don't take up much space. The data we'll cut is anything extraneous in the features. Have a look at the first feature:

In [8]:
geojson_ew['features'][0]

{'type': 'Feature',
 'properties': {'OBJECTID': 1,
  'LSOA11CD': 'E01000001',
  'LSOA11NM': 'City of London 001A',
  'LSOA11NMW': 'City of London 001A',
  'BNG_E': 532129,
  'BNG_N': 181625,
  'LONG': -0.09706,
  'LAT': 51.5181,
  'Shape__Area': 157794.481079102,
  'Shape__Length': 1685.39177789522,
  'GlobalID': 'b12173a3-5423-4672-a5eb-f152d2345f96'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-0.094744468765127, 51.5205961026855],
    [-0.095455174414778, 51.5154416842748],
    [-0.099722888517677, 51.5167693121822],
    [-0.098498304750799, 51.5205398973512],
    [-0.097265555652221, 51.5215848107683],
    [-0.094744468765127, 51.5205961026855]]]}}

We must retain the `type`, `properties` and `geometry` fields but can reduce the content within them. 

+ From properties, we only need to keep one identifier so we'll keep `LSOA11NM` and ditch the rest. 
+ From coordinates, we don't need as much precision as that so we'll round all values to fewer decimal places.

In [9]:
geojson_new = copy.deepcopy(geojson_ew)

In [10]:
for f, feature in enumerate(geojson_new['features']):
    LSOA_name = feature['properties']['LSOA11NM']
    LSOA_code = feature['properties']['LSOA11CD']
    # Manually update some bits that are inconsistent in the super generalised file:
    if LSOA_name == 'Shepway 014E':
        LSOA_name = 'Shepway 015A'
    if LSOA_name == 'Shepway 014F':
        LSOA_name = 'Shepway 015B'
    if LSOA_name == 'Shepway 014G':
        LSOA_name = 'Shepway 015C'
    if LSOA_name == 'Shepway 014H':
        LSOA_name = 'Shepway 015D'
        
    new_properties = {'LSOA11NM': LSOA_name, 'LSOA11CD': LSOA_code}

    # Get lists of coordinates, one list per separate polygon
    # in the LSOA.
    # (Have MultiPolygon e.g. when an LSOA on the mainland coastline
    # also contains an island, so there's a gap between areas.)
    lists_of_polygon_coordinates = feature['geometry']['coordinates']

    new_coords = []
    for lc, list_of_coords in enumerate(lists_of_polygon_coordinates):
        if feature['geometry']['type'] == 'MultiPolygon':
            # For MultiPolygons the coords are nested an extra time.
            list_of_coords = list_of_coords[0]
        new_coords_here = [
            [round(coords[0], 4), round(coords[1], 4)]
            for coords in list_of_coords
        ]
        if feature['geometry']['type'] == 'MultiPolygon':
            # For MultiPolygons the coords are nested an extra time.
            new_coords_here = [new_coords_here]

        new_coords.append(new_coords_here)

    # Overwrite the old data with the new:
    geojson_new['features'][f]['properties'] = new_properties
    geojson_new['features'][f]['geometry']['coordinates'] = new_coords

In [11]:
geojson_new['features'][0]

{'type': 'Feature',
 'properties': {'LSOA11NM': 'City of London 001A', 'LSOA11CD': 'E01000001'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-0.0947, 51.5206],
    [-0.0955, 51.5154],
    [-0.0997, 51.5168],
    [-0.0985, 51.5205],
    [-0.0973, 51.5216],
    [-0.0947, 51.5206]]]}}

Write new geojson to file:

In [12]:
# Save as a .geojson:
save_name = 'LSOA_V3_reduced.geojson'
with open(f'./{save_name}', 'w', encoding='utf-8') as f:
    json.dump(geojson_new, f, ensure_ascii=False)